In [1]:
import poppy
import jax
import jax.numpy as jnp
import functools
import galois
import numpy as np

In [2]:
p = 6661
n = 3
f = poppy.field( p, n )
f

field order 295541382781.

In [3]:
p = 31
n = 3
q = p ** n
f = poppy.field( p, n )
d = 2
a = poppy.random( ( d, d ), f, poppy.SEED + 0 )
b = poppy.random( ( d, d ), f, poppy.SEED + 1 )
c = poppy.random( ( 2, ), f, poppy.SEED + 1 )
a

array shape (2, 2).
field order 29791.

In [6]:
ai = a.inv( )
ar = a.reciprocal( )
bi = b.inv( )
ai, bi, ar.proj( )

(array shape (2, 2).
 field order 29791.,
 array shape (2, 2).
 field order 29791.,
 Array([[ 1564,  4530],
        [17864, 11457]], dtype=int64))

In [7]:
print( a.lift[ 0, : 10 ] )
( a - a @ ai @ a @ bi @ b ).lift

[ 8 16  7 20  2 13]


Array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [10]:
p = 31
n = 5
q = p ** n
f = poppy.field( p, n )
d = 3
a = poppy.random( ( d, d ), f, poppy.SEED + 0 )
r = a.reciprocal( )
r.proj( )

Array([[ 4054351, 17238574, 15971513],
       [ 1345567,  1266102, 23484550],
       [22372463, 13434895,  2775609]], dtype=int64)

In [11]:
GF = galois.Field( q )
ag = GF( np.array( a.proj( ) ) )
np.reciprocal( ag )

GF([[ 4054351, 17238574, 15971513],
    [ 1345567,  1266102, 23484550],
    [22372463, 13434895,  2775609]], order=31^5)

In [24]:
%timeit ap.trace( ).lift.block_until_ready( )

41.3 µs ± 474 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [25]:
an = np.array( a )
%timeit np.trace( an )

3.64 µs ± 55.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [26]:
ag = GF( np.array( a ) )
%timeit np.trace( ag )

76.2 µs ± 545 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
N  = 10000
d  = 2
p  = 7
p1 = 13
p2 = 17
n  = 6
q  = p ** n
f  = poppy.field( p,  n )
F  = poppy.field( p1, d )
FF = poppy.field( p2, n )
a  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 0 ), ( N, F.n, F.n ), 0,       q, dtype = jnp.int64 )
AA  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ),  ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
BB  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ),  ( F.n * FF.n, F.n * FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
A  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 1 ),  ( FF.n, FF.n ), 0, 2 ** 32, dtype = jnp.int64 )
B  = jax.random.randint( jax.random.PRNGKey( poppy.SEED + 2 ),  ( FF.n, FF.n ), 0, 2 ** 32, dtype = jnp.int64 )

@jax.jit
def poppyhash( a ):
    b = poppy.proj( AA @ a + BB, F )
    c = poppy.proj( A @ b + B, FF )
    return c % ( 4 * N )

liftbatch = jax.vmap( poppy.lift, in_axes = ( 0, None ) )
hashbatch = jax.vmap( poppyhash )

ap = liftbatch( a, f )
h  = hashbatch( ap ).squeeze( )
au = jnp.unique( a, axis = 0 )
hu = jnp.unique( h )
len( au ), len( hu ), F.q, FF.q

(10000, 8908, 169, 24137569)